In [1]:
import scipy.sparse.linalg as spla
import scipy.sparse as sp
import numpy as np
import matplotlib.pyplot as plt
import time
from collections import namedtuple, Counter
from scipy.optimize import curve_fit

# CODES

## EXACT DIAGONALIZATION

In [4]:
# Define operators
def clock_operators(q):
    """
    Define clock model operators for q-state system.
    
    Parameters:
    q: number of discrete states

    Returns:
    Clock and shift operator.
    """
    omega =  np.exp((2j * np.pi) / q)
    C = sp.csr_matrix(np.diag([omega ** m for m in range(q)]))
    S = sp.csr_matrix(np.roll(np.eye(q), shift=-1, axis=1))
    
    return C, S

def apply_operator_at_site(O, i, L, q):
    """
    Places an operator O at site i in an L-site q-state system.

    Parameters:
    O: operator to apply
    i: state index
    L: total number of sites
    q: number of discrete states per site

    Returns:
    The Kronecker product representing the operator acting on the full system.
    """
    L = int(L)
    if not (0 <= i < L):
        raise IndexError(f"Invalid site index i={i}. Must be between 0 and {L-1}")

    I_q = sp.identity(q, format="csr")
    op_list = [I_q] * L  # List of identity operators
    op_list[i] = O  # Apply operator at correct site

    # Compute the full Kronecker product
    full_operator = op_list[0]
    for op in op_list[1:]:
        full_operator = sp.kron(full_operator, op, format="csr")

    return full_operator

# Define Hamiltonian
def clock_hamiltonian(q, J, h, L, periodic = False):
    """
    Parameters:
    q: number of discrete states
    C: clock operator
    S: shift operator
    J: coupling constant
    h: external field strength
    L: number of sites
    """
    L = int(L)

    C, S = clock_operators(q)
    H = sp.csr_matrix((q**L, q**L))

    for i in range(1, L):
        H += (-J) * ((apply_operator_at_site(S, i-1, L, q) @ apply_operator_at_site(S.H, i, L, q)) 
             + (apply_operator_at_site(S.H, i-1, L, q) @ apply_operator_at_site(S, i, L, q)))
        H += (-J * (h / (2 * J))) * (apply_operator_at_site(C, i-1, L, q) + apply_operator_at_site(C.H, i-1, L, q) 
                                     + apply_operator_at_site(C, i, L, q) + apply_operator_at_site(C.H, i, L, q))

    if periodic:
        H += (-J) * ((apply_operator_at_site(S, L-1, L, q) @ apply_operator_at_site(S.H, 0, L, q)) 
                                 + (apply_operator_at_site(S.H, L-1, L, q) @ apply_operator_at_site(S, 0, L, q)))
    H += (-J * (h / (2 * J))) * (apply_operator_at_site(C, 0, L, q) + apply_operator_at_site(C.H, 0, L, q) 
                                 + apply_operator_at_site(C, L-1, L, q) + apply_operator_at_site(C.H, L-1, L, q))
    return H

# Ground-state energy density
def E0_per_site(q, J, h, L):
    H = clock_hamiltonian(q, J, h, L)
    eigval = spla.eigsh(H, k=1, which='SA', return_eigenvectors=False)  # Use eigsh for real Hermitian
    E0 = np.real(eigval[0])
    return E0 / L

# Energy gap per site
def E0_and_E1_per_site(q, J, h, L):
    H = clock_hamiltonian(q, J, h, L)
    eigval = spla.eigsh(H, k=2, which='SA', return_eigenvectors=False)  # Use eigsh for real Hermitian
    E0 = np.real(eigval[0])
    E1 = np.real(eigval[1])
    return np.abs(E1 - E0) / L

In [5]:
E0_and_E1_per_site(q=3, J=1.0, h=1.0, L=10)

0.04949269388393347

## INFINITE-SYSTEM DMRG

In [7]:
Block = namedtuple("Block", ["length", "basis_size", "operator_dict"])
EnlargedBlock = namedtuple("EnlargedBlock", ["length", "basis_size", "operator_dict"])

def is_valid_block(block):
    for op in block.operator_dict.values():
        if op.shape[0] != block.basis_size or op.shape[1] != block.basis_size:
            return False
    return True

is_valid_enlarged_block = is_valid_block

# Define operators
def clock_operators(q):
    """
    Define clock model operators for q-state system.
    """
    if not isinstance(q, (int, np.integer)):
        raise TypeError(f"Expected integer for q, got {type(q)}")

    omega = np.exp(2j * np.pi / q)
    C = np.diag([omega ** n for n in range(q)])
    S = sp.lil_matrix((q, q), dtype=complex)
    for i in range(q - 1):
        S[i, i + 1] = 1
    S[q - 1, 0] = 1
    return sp.csr_matrix(C), sp.csr_matrix(S)

# Single-site Hamiltonian
def H1(q, h, J):
    C, S = clock_operators(q)
    
    return - h * (C + C.getH())

def initialize_block(q, J, h):
    return Block(length=1, basis_size=q, operator_dict={
        "H": H1(q, h, J),
        "conn_C": clock_operators(q)[0],
        "conn_S": clock_operators(q)[1],
    })

def enlarge_block(block, q, J, h):
    """Enlarges the given Block by adding a single site."""
    mblock = block.basis_size
    o = block.operator_dict

    # Single-site Hamiltonian term
    H_single = sp.kron(sp.identity(mblock), H1(q, h, J))  

    # Neighbor interaction term (only between last site and new site)
    H_interaction = -J * (sp.kron(o["conn_S"], clock_operators(q)[1].conj().T) + sp.kron(o["conn_S"].conj().T, clock_operators(q)[1]))

    # New enlarged Hamiltonian
    enlarged_operator_dict = {
        "H": sp.kron(o["H"], sp.identity(q)) + H_single + H_interaction,
        "conn_C": (sp.kron(sp.identity(mblock), clock_operators(q)[0])),
        "conn_S": (sp.kron(sp.identity(mblock), clock_operators(q)[1])),
    }

    return EnlargedBlock(length=(block.length + 1), basis_size=(block.basis_size * q), operator_dict=enlarged_operator_dict)

def rotate_and_truncate(operator, transformation_matrix):
    """Transforms the operator to the new (possibly truncated) basis given by
    `transformation_matrix`.
    """
    return transformation_matrix.conj().transpose().dot(operator.dot(transformation_matrix))

def single_dmrg_step(sys, env, q, J, h, m):
    """Performs a single DMRG step for the q-state clock model."""
    sys_enl = enlarge_block(sys, q, J, h)
    env_enl = sys_enl if sys is env else enlarge_block(env, q, J, h)

    # Construct the superblock Hamiltonian
    superblock_hamiltonian = (
    sp.kron(sys_enl.operator_dict["H"], sp.identity(env_enl.basis_size)) +
    sp.kron(sp.identity(sys_enl.basis_size), env_enl.operator_dict["H"]) -J *
    (sp.kron(sys_enl.operator_dict["conn_S"], env_enl.operator_dict["conn_S"].getH()) + sp.kron(sys_enl.operator_dict["conn_S"].getH(), env_enl.operator_dict["conn_S"])
    ))
    (energy,), psi0 = spla.eigsh(superblock_hamiltonian, k=1, which="SA")

    psi0 = psi0.reshape([sys_enl.basis_size, -1], order="C")
    rho = np.dot(psi0, psi0.conjugate().transpose())

    evals, evecs = np.linalg.eigh(rho)
    possible_eigenstates = []
    for eval, evec in zip(evals, evecs.transpose()):
        possible_eigenstates.append((eval, evec))
    possible_eigenstates.sort(reverse=True, key=lambda x: x[0])  # largest eigenvalue first

    # Build the transformation matrix from the `m` overall most significant
    # eigenvectors.
    my_m = min(len(possible_eigenstates), m)
    transformation_matrix = np.zeros((sys_enl.basis_size, my_m), dtype='d', order='F')
    for i, (eval, evec) in enumerate(possible_eigenstates[:my_m]):
        transformation_matrix[:, i] = evec

    truncation_error = 1 - sum([x[0] for x in possible_eigenstates[:my_m]])
    print("truncation error:", truncation_error)

    # Rotate and truncate each operator.
    new_operator_dict = {}
    for name, op in sys_enl.operator_dict.items():
        new_operator_dict[name] = rotate_and_truncate(op, transformation_matrix)

    newblock = Block(length=sys_enl.length,
                     basis_size=my_m,
                     operator_dict=new_operator_dict)

    return newblock, energy, evals

def graphic(sys_block, env_block, sys_label="l"):
    """Returns a graphical representation of the DMRG step we are about to
    perform, using '=' to represent the system sites, '-' to represent the
    environment sites, and '**' to represent the two intermediate sites.
    """
    assert sys_label in ("l", "r")
    graphic = ("=" * sys_block.length) + "**" + ("-" * env_block.length)
    if sys_label == "r":
        # The system should be on the right and the environment should be on
        # the left, so reverse the graphic.
        graphic = graphic[::-1]
    return graphic

# Infinite-system DMRG
def infinite_system_algorithm(q, J, h, L, m):
    start_time = time.time()
    block = initialize_block(q, J, h)

    while 2 * block.length < L:  # Ensures final system size is L
        print("L =", block.length * 2 + 2)
        block, energy, _ = single_dmrg_step(block, block, q, J, h, m)
        print("E/L =", energy / (block.length * 2))
        print(f"Total runtime: {time.time() - start_time:.2f} seconds\n")

In [8]:
# Modified code to make plotting easier

def single_dmrg_step_list(sys, env, q, J, h, m):
    """Performs a single DMRG step for the q-state clock model."""
    sys_enl = enlarge_block(sys, q, J, h)
    env_enl = sys_enl if sys is env else enlarge_block(env, q, J, h)

    # Construct the superblock Hamiltonian
    superblock_hamiltonian = (
    sp.kron(sys_enl.operator_dict["H"], sp.identity(env_enl.basis_size)) +
    sp.kron(sp.identity(sys_enl.basis_size), env_enl.operator_dict["H"]) -J *
    (sp.kron(sys_enl.operator_dict["conn_S"], env_enl.operator_dict["conn_S"].getH()) + sp.kron(sys_enl.operator_dict["conn_S"].getH(), env_enl.operator_dict["conn_S"])
    ))
    (energy,), psi0 = spla.eigsh(superblock_hamiltonian, k=1, which="SA")

    psi0 = psi0.reshape([sys_enl.basis_size, -1], order="C")
    rho = np.dot(psi0, psi0.conjugate().transpose())

    evals, evecs = np.linalg.eigh(rho)
    possible_eigenstates = []
    for eval, evec in zip(evals, evecs.transpose()):
        possible_eigenstates.append((eval, evec))
    possible_eigenstates.sort(reverse=True, key=lambda x: x[0])  # largest eigenvalue first

    # Build the transformation matrix from the `m` overall most significant
    # eigenvectors.
    my_m = min(len(possible_eigenstates), m)
    transformation_matrix = np.zeros((sys_enl.basis_size, my_m), dtype='d', order='F')
    for i, (eval, evec) in enumerate(possible_eigenstates[:my_m]):
        transformation_matrix[:, i] = evec

    truncation_error = 1 - sum([x[0] for x in possible_eigenstates[:my_m]])

    # Rotate and truncate each operator.
    new_operator_dict = {}
    for name, op in sys_enl.operator_dict.items():
        new_operator_dict[name] = rotate_and_truncate(op, transformation_matrix)

    newblock = Block(length=sys_enl.length,
                     basis_size=my_m,
                     operator_dict=new_operator_dict)

    return newblock, energy, evals


def infinite_system_algorithm_list(q, J, h, L, m):
    start_time = time.time()
    block = initialize_block(q, J, h)

    E0_list = []
    while 2 * block.length < L:  # Ensures final system size is L
        block, energy, _ = single_dmrg_step_list(block, block, q, J, h, m)
        E0_list.append(energy / (block.length * 2))
    return E0_list

In [9]:
# Obtains evals from DMRG to find the entanglement spectrum
def entanglement(q, J, h, L, m):
    block = initialize_block(q, J, h)

    entanglement_spectrum = []
    while 2 * block.length < L:  # Ensures final system size is L
        block, energy, evals = single_dmrg_step(block, block, q, J, h, m)
        entanglement_spectrum.append(evals)
    return entanglement_spectrum

## FINITE-SYSTEM DMRG

In [11]:
# Finite-system DMRG
def finite_system_algorithm(q, J, h, L, m_warmup, m_sweep_list):
    assert L % 2 == 0  # require that L is an even number

    start_time = time.time()
    # To keep things simple, this dictionary is not actually saved to disk, but
    # we use it to represent persistent storage.
    block_disk = {}  # "disk" storage for Block objects

    # Use the infinite system algorithm to build up to desired size.  Each time
    # we construct a block, we save it for future reference as both a left
    # ("l") and right ("r") block, as the infinite system algorithm assumes the
    # environment is a mirror image of the system.
    block = initialize_block(q, J, h)
    block_disk["l", block.length] = block
    block_disk["r", block.length] = block
    while 2 * block.length < L:
        # Perform a single DMRG step and save the new Block to "disk"
        print(graphic(block, block))
        block, energy, _ = single_dmrg_step(block, block, q, J, h, m=m_warmup)
        print("E/L =", energy / (block.length * 2))
        block_disk["l", block.length] = block
        block_disk["r", block.length] = block

    # Now that the system is built up to its full size, we perform sweeps using
    # the finite system algorithm.  At first the left block will act as the
    # system, growing at the expense of the right block (the environment), but
    # once we come to the end of the chain these roles will be reversed.
    sys_label, env_label = "l", "r"
    sys_block = block; del block  # rename the variable
    for m in m_sweep_list:
        while True:
            # Load the appropriate environment block from "disk"
            env_block = block_disk[env_label, L - sys_block.length - 2]
            if env_block.length == 1:
                # We've come to the end of the chain, so we reverse course.
                sys_block, env_block = env_block, sys_block
                sys_label, env_label = env_label, sys_label

            # Perform a single DMRG step.
            print(graphic(sys_block, env_block, sys_label))
            sys_block, energy, _ = single_dmrg_step(sys_block, env_block, q, J, h, m=m)

            print("E/L =", energy / L)

            # Save the block from this step to disk.
            block_disk[sys_label, sys_block.length] = sys_block

            # Check whether we just completed a full sweep.
            if sys_label == "l" and 2 * sys_block.length == L:
                break  # escape from the "while True" loop
        end_time = time.time()
        print(f"Total runtime: {end_time - start_time:.2f} seconds")
        print("")
    return energy / L

In [12]:
# Modified code to make plotting easier
def finite_system_algorithm_list(q, J, h, L, m_warmup, m_sweep_list):
    assert L % 2 == 0  # require that L is an even number

    start_time = time.time()
    E0_list = []
    # To keep things simple, this dictionary is not actually saved to disk, but
    # we use it to represent persistent storage.
    block_disk = {}  # "disk" storage for Block objects

    # Use the infinite system algorithm to build up to desired size.  Each time
    # we construct a block, we save it for future reference as both a left
    # ("l") and right ("r") block, as the infinite system algorithm assumes the
    # environment is a mirror image of the system.
    block = initialize_block(q, J, h)
    block_disk["l", block.length] = block
    block_disk["r", block.length] = block
    while 2 * block.length < L:
        # Perform a single DMRG step and save the new Block to "disk"
        block, energy, _ = single_dmrg_step_list(block, block, q, J, h, m=m_warmup)
        block_disk["l", block.length] = block
        block_disk["r", block.length] = block

    # Now that the system is built up to its full size, we perform sweeps using
    # the finite system algorithm.  At first the left block will act as the
    # system, growing at the expense of the right block (the environment), but
    # once we come to the end of the chain these roles will be reversed.
    sys_label, env_label = "l", "r"
    sys_block = block; del block  # rename the variable
    for m in m_sweep_list:
        while True:
            # Load the appropriate environment block from "disk"
            env_block = block_disk[env_label, L - sys_block.length - 2]
            if env_block.length == 1:
                # We've come to the end of the chain, so we reverse course.
                sys_block, env_block = env_block, sys_block
                sys_label, env_label = env_label, sys_label

            # Perform a single DMRG step.
            sys_block, energy, _ = single_dmrg_step_list(sys_block, env_block, q, J, h, m=m)

            # Save the block from this step to disk.
            block_disk[sys_label, sys_block.length] = sys_block

            # Check whether we just completed a full sweep.
            if sys_label == "l" and 2 * sys_block.length == L:
                break  # escape from the "while True" loop
        end_time = time.time()
    E0_list.append(energy/L)
    return E0_list

In [13]:
# Modified code to find energy gap per site
def single_dmrg_step_gap(sys, env, q, J, h, m):
    """Performs a single DMRG step for the q-state clock model."""
    sys_enl = enlarge_block(sys, q, J, h)
    env_enl = sys_enl if sys is env else enlarge_block(env, q, J, h)

    # Construct the superblock Hamiltonian
    superblock_hamiltonian = (
    sp.kron(sys_enl.operator_dict["H"], sp.identity(env_enl.basis_size)) +
    sp.kron(sp.identity(sys_enl.basis_size), env_enl.operator_dict["H"]) -J *
    (sp.kron(sys_enl.operator_dict["conn_S"], env_enl.operator_dict["conn_S"].getH()) + sp.kron(sys_enl.operator_dict["conn_S"].getH(), env_enl.operator_dict["conn_S"])
    ))
    energy, psi0 = spla.eigsh(superblock_hamiltonian, k=2, which="SA")
    
    # return sys_enl, energy

    # Call ARPACK to find the superblock ground state.  ("SA" means find the
    # "smallest in amplitude" eigenvalue.)
    # (energy,), psi0 = eigsh(superblock_hamiltonian, k=1, which="SA")

    # Construct the reduced density matrix of the system by tracing out the
    # environment
    #
    # We want to make the (sys, env) indices correspond to (row, column) of a
    # matrix, respectively.  Since the environment (column) index updates most
    # quickly in our Kronecker product structure, psi0 is thus row-major ("C
    # style").
    psi0 = psi0[:,0].reshape([sys_enl.basis_size, -1], order="C")
    rho = np.dot(psi0, psi0.conjugate().transpose())


    # Diagonalize the reduced density matrix and sort the eigenvectors by
    # eigenvalue.
    evals, evecs = np.linalg.eigh(rho)
    possible_eigenstates = []
    for eval, evec in zip(evals, evecs.transpose()):
        possible_eigenstates.append((eval, evec))
    possible_eigenstates.sort(reverse=True, key=lambda x: x[0])  # largest eigenvalue first

    # Build the transformation matrix from the `m` overall most significant
    # eigenvectors.
    my_m = min(len(possible_eigenstates), m)
    transformation_matrix = np.zeros((sys_enl.basis_size, my_m), dtype='d', order='F')
    for i, (eval, evec) in enumerate(possible_eigenstates[:my_m]):
        transformation_matrix[:, i] = evec

    #truncation_error = 1 - sum([x[0] for x in possible_eigenstates[:my_m]])
    #print("truncation error:", truncation_error)

    # Rotate and truncate each operator.
    new_operator_dict = {}
    for name, op in sys_enl.operator_dict.items():
        new_operator_dict[name] = rotate_and_truncate(op, transformation_matrix)

    newblock = Block(length=sys_enl.length,
                     basis_size=my_m,
                     operator_dict=new_operator_dict)

    return newblock, energy, evals

def finite_system_algorithm_gap(q, J, h, L, m_warmup, m_sweep_list):
    assert L % 2 == 0  # require that L is an even number

    start_time = time.time
    energy_gap = []
    # To keep things simple, this dictionary is not actually saved to disk, but
    # we use it to represent persistent storage.
    block_disk = {}  # "disk" storage for Block objects

    # Use the infinite system algorithm to build up to desired size.  Each time
    # we construct a block, we save it for future reference as both a left
    # ("l") and right ("r") block, as the infinite system algorithm assumes the
    # environment is a mirror image of the system.
    block = initialize_block(q, J, h)
    block_disk["l", block.length] = block
    block_disk["r", block.length] = block
    while 2 * block.length < L:
        # Perform a single DMRG step and save the new Block to "disk"
        #print(graphic(block, block))
        block, energy, _ = single_dmrg_step_gap(block, block, q, J, h, m=m_warmup)
        #print("E/L =", energy / (block.length * 2))
        block_disk["l", block.length] = block
        block_disk["r", block.length] = block

    # Now that the system is built up to its full size, we perform sweeps using
    # the finite system algorithm.  At first the left block will act as the
    # system, growing at the expense of the right block (the environment), but
    # once we come to the end of the chain these roles will be reversed.
    sys_label, env_label = "l", "r"
    sys_block = block; del block  # rename the variable
    for m in m_sweep_list:
       while True:
            # Load the appropriate environment block from "disk"
            env_block = block_disk[env_label, L - sys_block.length - 2]
            if env_block.length == 1:
                # We've come to the end of the chain, so we reverse course.
                sys_block, env_block = env_block, sys_block
                sys_label, env_label = env_label, sys_label

            # Perform a single DMRG step.
            #print(graphic(sys_block, env_block, sys_label))
            sys_block, energy, evals = single_dmrg_step_gap(sys_block, env_block, q, J, h, m=m)

            #print("E/L =", energy / L)

            # Save the block from this step to disk.
            block_disk[sys_label, sys_block.length] = sys_block

            # Check whether we just completed a full sweep.
            if sys_label == "l" and 2 * sys_block.length == L:
                break  # escape from the "while True" loop
    end_time = time.time()
    energy_gap.append(np.abs((energy[1]-energy[0])/L))
        #print(f"Total runtime: {end_time - start_time:.2f} seconds")
        #print("")
    return energy_gap      

# PLOTS

## GROUND-STATE ENERGY DENSITY

### FINITE-SIZE SCALING OF GROUND-STATE ENERGY DENSITY (EXACT DIAGONALIZATION)

### FINITE-SIZE SCALING OF GROUND-STATE ENERGY DENSITY (INFINITE-SYSTEM DMRG)

### FINITE-SIZE SCALING OF GROUND-STATE ENERGY DENSITY (FINITE-SYSTEM DMRG)

### FINITE-SIZE SCALING OF GROUND-STATE ENERGY DENSITY (ED & DMRG)

### EXTRAPOLATION OF GROUND-STATE ENERGY DENSITY

## ENERGY GAP PER SITE

### FINITE-SIZE SCALING OF ENERGY GAP PER SITE (EXACT DIAGONALIZATION)

### FINITE-SIZE SCALING OF ENERGY GAP PER SITE (FINITE-SYSTEM DMRG)

### FINITE-SIZE SCALING OF ENERGY GAP PER SITE (ED & DMRG)

In [ ]:
q1 = 5
q2 = 6
m_warmup = 10
m_sweep_list = [10, 20, 30]
J = 1
h_values = np.array([0.0, 0.5, 1.0, 1.5, 2.0])
L_values1 = np.array([4, 6, 8])
L_values2 = np.array([6, 8, 10, 12, 14, 16])

start_time = time.time()

inv_L1 = 1 / L_values1  
inv_L2 = 1/ L_values2
exact1 = np.zeros((len(h_values), len(L_values1)))  # Store energies as 2D array
exact2 = np.zeros((len(h_values), len(L_values1)))

# Compute energy density using finite-system DMRG
dmrg1 = [[] for _ in h_values]
dmrg2 = [[] for _ in h_values]

# Compute energy density using exact diagonalization
for i, h in enumerate(h_values):
    for j, L in enumerate(L_values1):
        exact1[i, j] = (E0_and_E1_per_site(q1, J, h, L))    
        exact2[i, j] = (E0_and_E1_per_site(q2, J, h, L))

for i, h in enumerate(h_values):
    for L in L_values2:
        # Compute the energy values for the current (h, L) combination
        energies1 = finite_system_algorithm_gap(q1, J, h, L, m_warmup, m_sweep_list)
        energies2 = finite_system_algorithm_gap(q2, J, h, L, m_warmup, m_sweep_list)
        # Append the energies to the corresponding list for this h
        dmrg1[i].extend(energies1)
        dmrg2[i].extend(energies2)

end_time = time.time()
print(f"Total runtime: {end_time - start_time:.2f} seconds")

C:\Users\Tin Tin\AppData\Local\Temp\ipykernel_12376\901663880.py:45: ComplexWarning: Casting complex values to real discards the imaginary part
  transformation_matrix[:, i] = evec


In [ ]:
inv_L1 = 1 / L_values1  # Inverse system size (x-axis)
inv_L2 = 1/ L_values2

exact_handles1 = []
exact_handles2 = []
dmrg_handles1 = []
dmrg_handles2 = []
exact_markers = ['o', 'p', 'D', '^', 'v']
dmrg_markers = ['o', 'p', 'D', '^', 'v']

fig, axs = plt.subplots(1, 2, figsize = (24, 10))
for i, h in enumerate(h_values):
    scatter1 = axs[0].scatter(inv_L1, exact1[i], marker=exact_markers[i % len(exact_markers)], s=140, label=f"h = {h}", facecolors='none', edgecolors="black")
    scatter2 = axs[0].scatter(inv_L2, dmrg1[i], marker=dmrg_markers[i % len(exact_markers)], s=140, label=f"h = {h}", alpha=0.45)

    scatter3 = axs[1].scatter(inv_L1, exact2[i], marker=exact_markers[i % len(exact_markers)], s=140, label=f"h = {h}", facecolors='none', edgecolors="black")
    scatter4 = axs[1].scatter(inv_L2, dmrg2[i], marker=dmrg_markers[i % len(exact_markers)], s=140, label=f"h = {h}", alpha=0.45)
    
    exact_handles1.append(scatter1)
    dmrg_handles1.append(scatter2)
    exact_handles2.append(scatter3)
    dmrg_handles2.append(scatter4)

legend1 = axs[0].legend(exact_handles1, [f"h={h:.2f}" for h in h_values], loc='upper right', bbox_to_anchor=(1.0, 1), title="ED", fontsize=14)
axs[0].add_artist(legend1)
axs[0].legend(dmrg_handles1, [f"h={h:.2f}" for h in h_values], loc='upper right', bbox_to_anchor=(1.0, 0.75), title="DMRG", fontsize=14)
axs[0].grid(True)
axs[0].set_xlim(0.05, 0.3)
axs[0].set_ylim(-0.05, 0.4)
axs[0].set_xlabel("$1/L$", fontsize=18)
axs[0].set_ylabel("$E_{0}/L$", fontsize=18)

legend2 = axs[1].legend(exact_handles2, [f"h={h:.2f}" for h in h_values], loc='upper right', bbox_to_anchor=(1.0, 1), title="ED", fontsize=14)
axs[1].add_artist(legend2)
axs[1].legend(dmrg_handles2, [f"h={h:.2f}" for h in h_values], loc='upper right', bbox_to_anchor=(1.0, 0.75), title="DMRG", fontsize=14)
axs[1].grid(True)
axs[1].set_xlim(0.05, 0.3)
axs[1].set_ylim(-0.05, 0.4)
axs[1].set_xlabel("$1/L$", fontsize=18)
axs[1].set_ylabel("$E_{0}/L$", fontsize=18)

plt.show()

end_time = time.time()
print(f"Total runtime: {end_time - start_time:.2f} seconds")

## ENTANGLEMENT SPECTRUM